### Load and Prepare Data

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Load in data: df
df = pd.read_csv('https://s3.amazonaws.com/clcarverloans/data/df_cleaned.csv')

In [4]:
# Remove usesless rows: df_select
cols = ['Borrower_Genders', 'Country Code', 'Loan Amount', 'Lender_Term',
       'Repayment_Interval', 'Distribution_Model', 'Sector', 'Activity','Delinquent']
df_select = df[cols]

# One_hot_encode categorical columns: df_dummies
df_dummies = pd.get_dummies(df_select)

# Select features and target: X, y
y = df_dummies.pop('Delinquent')
X = df_dummies

In [6]:
# Split data into train and test set: train_set, test_set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Create Trade-off Metric

First off, we should know what the average principal per loan is

In [11]:
average_loan_amount = df['Loan Amount'].mean()
print('Average Loan Size is: $', average_loan_amount)

Average Loan Size is: $ 770.9566877046869


Next, we should know average interest rate of the loan. According to Kiva's Q&A section the field partners charge around 36%

In [14]:
average_interest_rate = 0.36
print('Average Interest Rate: {}%'.format(int(average_interest_rate*100)))

Average Interest Rate: 36%


In [15]:
average_profit = average_loan_amount * average_interest_rate
print('Average Profit Per Loan: $', average_profit)

Average Profit Per Loan: $ 277.5444075736873


This means, on average, the banks make around $278 dollars per loan. The resulting cost benefit matrix would look like so: